In [1]:
import pandas as pd
import os
import re
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.snowball import SnowballStemmer
from difflib import SequenceMatcher



In [2]:
text_clustered = pd.read_csv('df_clusters.csv')

In [3]:
text_clustered.tail()

,Unnamed: 0,texts,key_words,clusters
861,861,introduction many researchers investigating is...,"['sarah', 'equity', 'positioning', 'computer',...",2
862,862,developing historical thinking pbl class suppo...,"['pbl', 'instructor', 'historical', 'scaffoldi...",2
863,863,keywords attentional anchor coordination eye t...,"['gaze', 'parallel', 'reflective', 'action', '...",3
864,864,introduction series papers paul kirschner kirs...,"['epistemology', 'science', 'scientific', 'the...",2
865,865,beyond copy room collaboration case study onli...,"['professional', 'discourse', 'facebook', 'gro...",0


In [4]:
groups = text_clustered.groupby('clusters')

In [5]:
len(groups.get_group(0))

219

In [6]:
len(groups.get_group(1))

246

In [7]:
len(groups.get_group(2))

223

In [8]:
len(groups.get_group(3))

178

In [9]:
rootdir = 'data/papers'

In [10]:
contents = []
file_ = []
i = 0
source = []
errors = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        file_.append(file[:-4])
        if 'txt' in file:
            i += 1
            path = os.path.join(subdir, file)
            with open(path) as file:
                try:
                    text = file.read()
                    contents.append(text[: text.find('\n\nReferences\n')])
                    source.append(subdir[len(rootdir):])
                except:
                    name, message, content = sys.exc_info()
                    errors.append(message)

In [11]:
stop_words = set(stopwords.words())
l = ['the','and','of','is','with','it','this','to','but','not','in','for','some','as','no','was','just','would']
for word in l : 
    stop_words.add(word)

In [12]:
tmp_content = []
tmp_files = []
for i,text_ in enumerate(contents):
    selection = text_[text_.find('Introduction'):text_.find('\n\nReferences\n')]
    if selection == '' and text_ != ' ':
        selection =text_[text_.find('Keywords'):text_.find('\n\nReferences\n')]
    if selection == '' and text_ != ' ':
        selection = text_[:text_.find('\n\nReferences\n')]
    if '...............................' in selection :
        selection = ''
    tmp_content.append(selection)


In [13]:
df = pd.DataFrame({'text':tmp_content, 'files': file_})

In [14]:
df = df[df['text']!= '']
df.reset_index(inplace=True)

In [15]:
df['text_from_extraction'] =  text_clustered['texts']

In [17]:
df['clusters'] = text_clustered['clusters']
df['keywors'] = text_clustered['key_words']


In [20]:
df.head()

,index,text,files,text_from_extraction,clusters,keywors
0,0,"Introduction\n\nOver the last two decades, the...",import2018_371,introduction last two decades increased focus ...,0,"['community', 'lab', 'scientists', 'genetics',..."
1,1,Introduction\n\nVideo data is one of the most ...,import2018_417,introduction video data one complex types data...,1,"['engineering design', 'engineering', 'macro',..."
2,2,Opening the Door to Algebra: The Role of Fract...,import2018_403,opening algebra role fraction knowledge algebr...,1,"['algebra', 'fractions', 'knowledge', 'th', 'y..."
3,3,"Introduction\n\nRecently, researchers have ide...",import2018_365,introduction recently researchers identified d...,2,"['maker', 'constructs', 'dimension', 'construc..."
4,4,"Keywords: mathematics, argumentation, collabor...",import2016full_78,keywords mathematics argumentation collaborati...,1,"['script', 'argumentation', 'mathematical', 's..."


In [19]:
df.to_csv('to_text_files.csv') 

In [36]:
tmp = pd.read_csv('references_nour.csv')
tmp.rename(columns={'Unnamed: 0':'keys', '0': 'titles'},inplace = True)

In [37]:
tmp.head()

,keys,Unnamed: 0.1,ref,file,ref_parsed,year,identifier,identifier_title
0,0,0,"Lave, J., & Wenger, E. (1991). Situated learni...",import2018_371,"Lave, J., & Wenger, E. (1991)",1991.0,"Lave, J., & Wenger, E. (1991). Situated learni...","Swanson, R. D., Hinojosa, L., & Polman, J. L. ..."
1,1,1,"Miles, M. B., Huberman, A. M., & Saldana, J. (...",import2018_371,"Miles, M. B., Huberman, A. M., & Saldana, J. (...",2013.0,"Miles, M. B., Huberman, A. M., & Saldana, J. (...","Swanson, R. D., Hinojosa, L., & Polman, J. L. ..."
2,2,2,"Pea, R. D., Gomez, L. M., Edelson, D. C., Fish...",import2018_371,"Pea, R. D., Gomez, L. M., Edelson, D. C., Fish...",1997.0,"Pea, R. D., Gomez, L. M., Edelson, D. C., Fish...","Swanson, R. D., Hinojosa, L., & Polman, J. L. ..."
3,3,3,"Rogoff, B. (1995). Observing sociocultural act...",import2018_371,"Rogoff, B. (1995)",1995.0,"Rogoff, B. (1995). Observing sociocultural act...","Swanson, R. D., Hinojosa, L., & Polman, J. L. ..."
4,4,4,"Shirk, J., Ballard, H., Wilderman, C., Phillip...",import2018_371,"Shirk, J., Ballard, H., Wilderman, C., Phillip...",2012.0,"Shirk, J., Ballard, H., Wilderman, C., Phillip...","Swanson, R. D., Hinojosa, L., & Polman, J. L. ..."


In [40]:
dict_matching = dict(zip(list(tmp['file']),list(tmp['identifier_title'])))

In [41]:
dict_matching

{'import2015full_105': 'Downton, M. P. (2015). “That’s What Everyone Else Is Saying…”: Collaborative Reflection-in-Action during Creative Activities In  Lindwall, O',
 'import2015full_122': 'Fong, C., Cober, R., Messina, R., Moher, T., Murray, J., Peebles, B., & Slotta, J. D. (2015). The 3R Orchestration Cycle: Fostering Multi-Modal Inquiry Discourse in a Scaffolded Inquiry Environment In  Lindwall, O',
 'import2015full_126': 'Hong, H., Chai, C. S., & Hung, G. (2015). Principle-Based Guidance to Foster Adaptive Teaching Practice In  Lindwall, O',
 'import2015full_136': "d'Apollonia, S., Kunicki, S., & Bronet, M. (2015). Connected Biology: A Usability Study of Web 2",
 'import2015full_146': 'Oshima, J., Oshima, R., & Fujita, W. (2015). A Multivocality Approach to Epistemic Agency in Collaborative Learning In  Lindwall, O',
 'import2015full_150': 'Davidsen, J. & Ryberg, T. (2015). “This Is the Size of One Meter”: Children’s Bodily-Material Collaboration and Understanding of Scale around 

In [42]:
df['title'] = df['files'].apply(lambda x : dict_matching[x])

In [43]:
df.head()

,index,text,files,text_from_extraction,clusters,keywors,title
0,0,"Introduction\n\nOver the last two decades, the...",import2018_371,introduction last two decades increased focus ...,0,"['community', 'lab', 'scientists', 'genetics',...","Swanson, R. D., Hinojosa, L., & Polman, J. L. ..."
1,1,Introduction\n\nVideo data is one of the most ...,import2018_417,introduction video data one complex types data...,1,"['engineering design', 'engineering', 'macro',...","Huang, J., Gomoll, A. S., Tolar, E., Silver, C..."
2,2,Opening the Door to Algebra: The Role of Fract...,import2018_403,opening algebra role fraction knowledge algebr...,1,"['algebra', 'fractions', 'knowledge', 'th', 'y...","Booth, J. L., Newton, K. J., Pendergast, L. L...."
3,3,"Introduction\n\nRecently, researchers have ide...",import2018_365,introduction recently researchers identified d...,2,"['maker', 'constructs', 'dimension', 'construc...","Cohen, J., Margulieux, L. E., Renken, M., Smit..."
4,4,"Keywords: mathematics, argumentation, collabor...",import2016full_78,keywords mathematics argumentation collaborati...,1,"['script', 'argumentation', 'mathematical', 's...","Vogel, F., Kollar, I., Ufer, S., Reiss, K., & ..."
